In [2]:
import pandas as pd
import pm4py

In [2]:
application_log = pm4py.read_xes('BPI Challenge 2017.xes')
offer_log = pm4py.read_xes('BPI Challenge 2017 - Offer log.xes')

parsing log, completed traces :: 100%|██████████| 42995/42995 [00:10<00:00, 4021.40it/s]


In [3]:
application_log[0]

{'attributes': {'LoanGoal': 'Existing loan takeover', 'ApplicationType': 'New credit', 'concept:name': 'Application_652823628', 'RequestedAmount': 20000.0}, 'events': [{'Action': 'Created', 'org:resource': 'User_1', 'concept:name': 'A_Create Application', 'EventOrigin': 'Application', 'EventID': 'Application_652823628', 'lifecycle:transition': 'complete', 'time:timestamp': datetime.datetime(2016, 1, 1, 9, 51, 15, 304000, tzinfo=datetime.timezone.utc)}, '..', {'Action': 'Deleted', 'org:resource': 'User_115', 'concept:name': 'W_Validate application', 'EventOrigin': 'Workflow', 'EventID': 'Workitem_2067229931', 'lifecycle:transition': 'ate_abort', 'time:timestamp': datetime.datetime(2016, 1, 14, 15, 49, 11, 423000, tzinfo=datetime.timezone.utc)}]}

In [4]:
df_application = pm4py.convert_to_dataframe(application_log)
df_offer = pm4py.convert_to_dataframe(offer_log)

In [7]:
df_application.to_csv('app_logs.csv')
df_offer.to_csv('offer_logs.csv')

In [3]:
df_application = pd.read_csv('app_logs.csv')
df_offer = pd.read_csv('offer_logs.csv')

In [4]:
df_application.loc[df_application['case:concept:name'] == 'Application_652823628']

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Obtained,User_17,W_Complete application,Workflow,Workitem_1875340971,start,2016-01-02 10:45:22.429000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Released,User_17,W_Complete application,Workflow,Workitem_1452291795,suspend,2016-01-02 10:49:28.816000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,statechange,User_52,A_Accepted,Application,ApplState_99568828,complete,2016-01-02 11:23:04.299000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Created,User_52,O_Create Offer,Offer,Offer_148581083,complete,2016-01-02 11:29:03.994000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,20000.0,44.0,True,498.29,True,979.0,20000.0,NaN


In [6]:
sorted(list(df_application['concept:name'].unique()))

['A_Accepted',
 'A_Cancelled',
 'A_Complete',
 'A_Concept',
 'A_Create Application',
 'A_Denied',
 'A_Incomplete',
 'A_Pending',
 'A_Submitted',
 'A_Validating',
 'O_Accepted',
 'O_Cancelled',
 'O_Create Offer',
 'O_Created',
 'O_Refused',
 'O_Returned',
 'O_Sent (mail and online)',
 'O_Sent (online only)',
 'W_Assess potential fraud',
 'W_Call after offers',
 'W_Call incomplete files',
 'W_Complete application',
 'W_Handle leads',
 'W_Personal Loan collection',
 'W_Shortened completion ',
 'W_Validate application']

In [8]:
declined_ids = list(df_application.loc[df_application['concept:name'] == 'A_Denied']['case:concept:name'].unique())
accepted_ids = list(df_application.loc[df_application['concept:name'] == 'A_Pending']['case:concept:name'].unique())
cancelled_ids = list(df_application.loc[df_application['concept:name'] == 'A_Cancelled']['case:concept:name'].unique())

In [9]:
len(declined_ids)

3752

In [ ]:
def create_datasets(df_application, current_event, target_event):
    """
    Encode original applications 
    """
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO: get all unique activities before target_event
    # TODO: get all unique resources before target_event
    # use abrove info to aggregate below

    for app_id in app_id_list:

        all_events = df_application.loc[df_application['case:concept:name'] == app_id]

        # aggregate rows of events and append the timestamp of the target_event

        # Encode each case attribute as a feature (or one-hot encode categorical case attributes)

        # For each numerical event attribute, apply an aggregation function (e.g. average) over the sequence of values taken by this attribute in the prefix
        # sum up time as one feature?

        # For each categorical event attribute, encode each possible value of that attribute as a numerical feature. 
        
        
        

In [31]:
list(df_application['case:concept:name'].unique())[:2]

['Application_652823628', 'Application_1691306052']

In [16]:
df_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202267 entries, 0 to 1202266
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype              
---  ------                 --------------    -----              
 0   Action                 1202267 non-null  object             
 1   org:resource           1202267 non-null  object             
 2   concept:name           1202267 non-null  object             
 3   EventOrigin            1202267 non-null  object             
 4   EventID                1202267 non-null  object             
 5   lifecycle:transition   1202267 non-null  object             
 6   time:timestamp         1202267 non-null  datetime64[ns, UTC]
 7   case:LoanGoal          1202267 non-null  object             
 8   case:ApplicationType   1202267 non-null  object             
 9   case:concept:name      1202267 non-null  object             
 10  case:RequestedAmount   1202267 non-null  float64            
 11  FirstWithdrawalAmount  4

In [21]:
len(list(df_offer['OfferID'].unique()))

42996

In [27]:
df_offer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193849 entries, 0 to 193848
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   Action                      193849 non-null  object             
 1   org:resource                193849 non-null  object             
 2   concept:name                193849 non-null  object             
 3   EventOrigin                 193849 non-null  object             
 4   EventID                     193849 non-null  object             
 5   lifecycle:transition        193849 non-null  object             
 6   time:timestamp              193849 non-null  datetime64[ns, UTC]
 7   case:concept:name           193849 non-null  object             
 8   case:MonthlyCost            193849 non-null  float64            
 9   case:Selected               193849 non-null  bool               
 10  case:ApplicationID          193849 non-null 

In [29]:
df_offer.loc[df_offer['case:concept:name'] == 'Offer_247135719']

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:concept:name,case:MonthlyCost,case:Selected,case:ApplicationID,case:FirstWithdrawalAmount,case:CreditScore,case:OfferedAmount,case:NumberOfTerms,case:Accepted,OfferID
0,Created,User_17,O_Create Offer,Offer,Offer_247135719,complete,2016-01-02 09:17:05.720000+00:00,Offer_247135719,201.76,False,Application_196483749,10000.0,0,10000.0,57,True,NaN
1,statechange,User_17,O_Created,Offer,OfferState_124849367,complete,2016-01-02 09:17:08.762000+00:00,Offer_247135719,201.76,False,Application_196483749,10000.0,0,10000.0,57,True,Offer_247135719
2,statechange,User_17,O_Sent (online only),Offer,OfferState_440662877,complete,2016-01-02 09:19:21.330000+00:00,Offer_247135719,201.76,False,Application_196483749,10000.0,0,10000.0,57,True,Offer_247135719
3,statechange,User_17,O_Cancelled,Offer,OfferState_591416028,complete,2016-01-02 09:21:26.034000+00:00,Offer_247135719,201.76,False,Application_196483749,10000.0,0,10000.0,57,True,Offer_247135719
